In [168]:

print('Please wait...')

# Importing packages:
from src.CleanFilter import *
from src.api import *
from src.mongodb import *
from src.input import *
from src.output import *

title()

# Importin data:
df = pd.read_csv('./input/companies_df.csv', low_memory = False)
cols = ['_id', 'name', 'founded_year', 'category_code','deadpooled_year', 'total_money_raised_USD', 'num_offices',
 'office_1_longitude','office_1_latitude', 'office_1_location', 'office_1_city','office_1_state_code',
 'office_1_country_code']
df = df[cols]

# Importing database and collection:
db, coll = connectCollection('companies','companies_cleaned')
companies = list(coll.find())

########################################################################################################

# Developers like to be near successful tech startups that have raised at least n (1 Million) dollars
# inputmoney = 1000000

input_money()
while True:
    inputmoney = input('n = ')
    try:
        inputmoney = int(inputmoney)
        break
    except ValueError:
        print('Please enter a valid integer')
        continue
print('Please wait...')

successful_tech_startups = list(coll.find({'$and':[{'$or':[
        {'category_code':'semiconductor'},{'category_code':'network_hosting'},{'category_code':'consulting'},
        {'category_code':'design'},{'category_code':'hardware'},{'category_code':'nanotech'},
        {'category_code':'mobile'},{'category_code':'games_video'},{'category_code':'cleantech'},
        {'category_code':'software'},{'category_code':'analytics'},{'category_code':'web'},
        {'category_code':'biotech'}]},
    {'deadpooled_year': np.nan},{'founded_year':{'$gte':1999}},{'total_money_raised_USD':{'$gte':inputmoney}}]}))

startups_and_near_companies = []
possible_offices_criterion_1 = []
for e in successful_tech_startups:
    near_companies = getCompaniesNear(e['office_1_location']['coordinates'][0],
                                      e['office_1_location']['coordinates'][1],2000)
    aux = []
    aux.append([e['_id'], e['name'], e['founded_year'], e['category_code'], e['total_money_raised'], 
          e['office_1_longitude'], e['office_1_latitude']])
    aux.append([near_companies[i] for i in range(len(near_companies)) if near_companies[i]['name'] != e['name']])    
    startups_and_near_companies.append(aux)
    for f in near_companies:
        if f['_id'] != e['name']:
            possible_offices_criterion_1.append(f['_id'])
possible_offices_c1 = list(set(possible_offices_criterion_1))

Please wait...

    ----------------------------------------------------------------------------------------------------
                                FIND THE PERFECT LOCATION FOR YOUR COMPANY              
    ----------------------------------------------------------------------------------------------------
    


    
        Developers like to be near successful tech startups that have raised at least (n) dollars.
    ----------------------------------------------------------------------------------------------------
    

        What amount of money should they have raised?
        Please enter a valid integer, without thousands separator.
        Example: For 1 million dollars, enter: 1000000

    
n = 1000000
Please wait...


In [170]:
len(possible_offices_c1)

4798

In [171]:

# Nobody in the company likes to have companies with more than n (10) years in a radius of 2 KM
# inputyears= 10

input_year()

while True:
    inputyears = input('n = ')
    try:
        inputyears = int(inputyears)
        break
    except ValueError:
        print('Please enter a valid integer')
        continue

print('Please wait...')

old_companies = list(coll.find({'$and':[{'deadpooled_year': np.nan},{'founded_year':{'$lte':2019-inputyears}}]}))

not_possible_offices_criterion_2 = []
for e in old_companies:
    near_companies = getCompaniesNear(e['office_1_location']['coordinates'][0],
                                      e['office_1_location']['coordinates'][1],2000)
    for f in near_companies:
        if f['_id'] != e['name']:
            not_possible_offices_criterion_2.append(f['_id'])

not_possible_offices_c2 = list(set(not_possible_offices_criterion_2))





        Nobody in the company likes to have companies with more than (n) years in a radius of 2 KM.
    ----------------------------------------------------------------------------------------------------
    

        How old can these companies be at most?
        Please enter a valid integer.
        Example: For 10 years, enter: 10
    
    
n = 12
Please wait...


In [172]:
len(not_possible_offices_c2)

7912

In [173]:
# Taking into account both criteria:
possible_offices_c1_c2 = valuesInCommon(possible_offices_c1, not_possible_offices_c2)
possible_offices_c1_c2_to_string = [str(e) for e in possible_offices_c1_c2]
possible_offices_c1_c2_coords = getOfficesCoords(possible_offices_c1_c2, companies)

In [174]:
len(possible_offices_c1_c2)

30

In [194]:
possible_offices_c1_c2

[ObjectId('52cdef7f4bab8bd67529bfb3'),
 ObjectId('52cdef7f4bab8bd67529c2d0'),
 ObjectId('52cdef7e4bab8bd67529bd0c'),
 ObjectId('52cdef7d4bab8bd6752995b1'),
 ObjectId('52cdef7e4bab8bd67529b279'),
 ObjectId('52cdef7d4bab8bd675299c4f'),
 ObjectId('52cdef7e4bab8bd67529afed'),
 ObjectId('52cdef7d4bab8bd67529993b'),
 ObjectId('52cdef7e4bab8bd67529bbec'),
 ObjectId('52cdef7f4bab8bd67529bda4'),
 ObjectId('52cdef7d4bab8bd67529a00e'),
 ObjectId('52cdef7d4bab8bd675299b49'),
 ObjectId('52cdef7c4bab8bd675298644'),
 ObjectId('52cdef7c4bab8bd675298232'),
 ObjectId('52cdef7d4bab8bd675299da8'),
 ObjectId('52cdef7d4bab8bd675298ce2'),
 ObjectId('52cdef7d4bab8bd675299115'),
 ObjectId('52cdef7f4bab8bd67529c040'),
 ObjectId('52cdef7d4bab8bd6752997bf'),
 ObjectId('52cdef7f4bab8bd67529c00d'),
 ObjectId('52cdef7d4bab8bd67529a171'),
 ObjectId('52cdef7d4bab8bd67529a0d1'),
 ObjectId('52cdef7c4bab8bd6752980be'),
 ObjectId('52cdef7d4bab8bd675298f4a'),
 ObjectId('52cdef7f4bab8bd67529c5b8'),
 ObjectId('52cdef7d4bab8b

In [175]:
# Account managers need to travel a lot (Airport < 20 km)
input_airport()
airports_df = pd.read_csv('./input/airports.csv', header=None,usecols=[1,2,3,4,6,7,12], 
                          names=['FacilityName', 'City', 'Country','3CharCode','Lat','Long','FacilityType'])
airports_df = airports_df[airports_df['FacilityType']=='airport']
airports_df.reset_index(drop=True, inplace=True)

distances = []
for e in possible_offices_c1_c2_coords:
    coords_1 = (e['coordinates'][1], e['coordinates'][0])
    dist = []
    for i in range(len(airports_df)):
        coords_2 = (airports_df.at[i,'Lat'],airports_df.at[i,'Long'])
        dist.append(geopy.distance.geodesic(coords_1, coords_2).km)
    distances.append(dist)    

office_airports = []
for i in range(len(distances)):
    aux = []
    aux.append(possible_offices_c1_c2[i])
    for j in range(len(distances[i])):
        if distances[i][j] <= 20:
            aux.append([airports_df.at[j,'FacilityName'],
                        distances[i][j],
                        airports_df.at[j,'Lat'],
                        airports_df.at[j,'Long']])
    office_airports.append(aux)

possible_offices_c1_c2_c3 = []
for e in office_airports:
    if len(e) >= 2:
        possible_offices_c1_c2_c3.append(e[0])

possible_offices_c1_c2_c3_to_string = [str(e) for e in possible_offices_c1_c2_c3]
possible_offices_c1_c2_c3_coords = getOfficesCoords(possible_offices_c1_c2_c3, companies)





                                Account managers need to travel a lot.
    ----------------------------------------------------------------------------------------------------
    

                            Please wait. We are collecting some information...
    


In [176]:
len(possible_offices_c1_c2_c3)

28

In [195]:
possible_offices_c1_c2_c3

[ObjectId('52cdef7f4bab8bd67529bfb3'),
 ObjectId('52cdef7d4bab8bd6752995b1'),
 ObjectId('52cdef7e4bab8bd67529b279'),
 ObjectId('52cdef7d4bab8bd675299c4f'),
 ObjectId('52cdef7e4bab8bd67529afed'),
 ObjectId('52cdef7d4bab8bd67529993b'),
 ObjectId('52cdef7e4bab8bd67529bbec'),
 ObjectId('52cdef7f4bab8bd67529bda4'),
 ObjectId('52cdef7d4bab8bd67529a00e'),
 ObjectId('52cdef7d4bab8bd675299b49'),
 ObjectId('52cdef7c4bab8bd675298644'),
 ObjectId('52cdef7c4bab8bd675298232'),
 ObjectId('52cdef7d4bab8bd675299da8'),
 ObjectId('52cdef7d4bab8bd675298ce2'),
 ObjectId('52cdef7d4bab8bd675299115'),
 ObjectId('52cdef7f4bab8bd67529c040'),
 ObjectId('52cdef7d4bab8bd6752997bf'),
 ObjectId('52cdef7f4bab8bd67529c00d'),
 ObjectId('52cdef7d4bab8bd67529a171'),
 ObjectId('52cdef7d4bab8bd67529a0d1'),
 ObjectId('52cdef7c4bab8bd6752980be'),
 ObjectId('52cdef7d4bab8bd675298f4a'),
 ObjectId('52cdef7f4bab8bd67529c5b8'),
 ObjectId('52cdef7d4bab8bd67529950e'),
 ObjectId('52cdef7c4bab8bd6752986aa'),
 ObjectId('52cdef7f4bab8b

In [177]:

# Executives like Starbucks A LOT. Ensure there's a starbucks not to far
input_starbucks()
starbucks_list = venuesListByQuery(possible_offices_c1_c2_c3_coords, possible_offices_c1_c2_c3,
                                  'starbucks', 1000)
cleaned_starbucks_list = cleanVenueList(starbucks_list)              
possible_offices_c1_c2_c3_c4 = [e[0] for e in cleaned_starbucks_list]
possible_offices_c1_c2_c3_c4_to_string = [str(e) for e in possible_offices_c1_c2_c3_c4]
possible_offices_c1_c2_c3_c4_coords = getOfficesCoords(possible_offices_c1_c2_c3_c4, companies)





                    Executives like Starbucks A LOT. Ensure there's a Starbucks not to far.
    ----------------------------------------------------------------------------------------------------
    

                            Please wait. We are collecting some information...
    


In [178]:
len(possible_offices_c1_c2_c3_c4_coords)

15

In [196]:
possible_offices_c1_c2_c3_c4

[ObjectId('52cdef7d4bab8bd6752995b1'),
 ObjectId('52cdef7e4bab8bd67529b279'),
 ObjectId('52cdef7d4bab8bd675299c4f'),
 ObjectId('52cdef7e4bab8bd67529afed'),
 ObjectId('52cdef7d4bab8bd67529993b'),
 ObjectId('52cdef7d4bab8bd67529a00e'),
 ObjectId('52cdef7d4bab8bd675299b49'),
 ObjectId('52cdef7d4bab8bd675299115'),
 ObjectId('52cdef7d4bab8bd6752997bf'),
 ObjectId('52cdef7d4bab8bd67529a171'),
 ObjectId('52cdef7d4bab8bd67529a0d1'),
 ObjectId('52cdef7c4bab8bd6752980be'),
 ObjectId('52cdef7d4bab8bd67529950e'),
 ObjectId('52cdef7c4bab8bd6752986aa'),
 ObjectId('52cdef7f4bab8bd67529c1ad')]

In [179]:

# The CEO is Vegan
input_vegan()
# vegan_list = venuesListByQuery(possible_offices_c1_c2_c3_c4_coords, possible_offices_c1_c2_c3_c4, 'vegan', 1000)
vegan_list = venuesListByCategory(possible_offices_c1_c2_c3_c4_coords, possible_offices_c1_c2_c3_c4,
             '4bf58dd8d48988d1d3941735', 1000)
cleaned_vegan_list = cleanVenueList(vegan_list)
possible_offices_c1_c2_c3_c4_c5 = [e[0] for e in cleaned_vegan_list]
possible_offices_c1_c2_c3_c4_c5_to_string = [str(e) for e in possible_offices_c1_c2_c3_c4_c5]
possible_offices_c1_c2_c3_c4_c5_coords = getOfficesCoords(possible_offices_c1_c2_c3_c4_c5, companies)





                                        The CEO is Vegan.
    ----------------------------------------------------------------------------------------------------
    

                            Please wait. We are collecting some information...
    


In [180]:
len(possible_offices_c1_c2_c3_c4_c5_coords)

5

In [197]:
possible_offices_c1_c2_c3_c4_c5

[ObjectId('52cdef7e4bab8bd67529afed'),
 ObjectId('52cdef7d4bab8bd675299115'),
 ObjectId('52cdef7d4bab8bd67529a0d1'),
 ObjectId('52cdef7c4bab8bd6752980be'),
 ObjectId('52cdef7c4bab8bd6752986aa')]

In [181]:
# All people in the company have between 25 and 40 years, give them some place to go to party
input_party()
party_list = venuesListByCategory(possible_offices_c1_c2_c3_c4_c5_coords, possible_offices_c1_c2_c3_c4_c5,
 '4bf58dd8d48988d11f941735', 1000)
cleaned_party_list = cleanVenueList(party_list)
possible_offices_c1_c2_c3_c4_c5_c6 = [e[0] for e in cleaned_party_list]
possible_offices_c1_c2_c3_c4_c5_c6_to_string = [str(e) for e in possible_offices_c1_c2_c3_c4_c5_c6]
possible_offices_c1_c2_c3_c4_c5_c6_coords = getOfficesCoords(possible_offices_c1_c2_c3_c4_c5_c6, companies)





        All people in the company have between 25 and 40 years, give them some place to go to party.
    ----------------------------------------------------------------------------------------------------
    

                            Please wait. We are collecting some information...
    


In [182]:
len(possible_offices_c1_c2_c3_c4_c5_c6_coords)

4

In [198]:
possible_offices_c1_c2_c3_c4_c5_c6

[ObjectId('52cdef7d4bab8bd675299115'),
 ObjectId('52cdef7d4bab8bd67529a0d1'),
 ObjectId('52cdef7c4bab8bd6752980be'),
 ObjectId('52cdef7c4bab8bd6752986aa')]

In [183]:

# 30% of the company have at least 1 child (Schools < 5 km)
input_school()
school_list = venuesListByCategory(possible_offices_c1_c2_c3_c4_c5_c6_coords, 
possible_offices_c1_c2_c3_c4_c5_c6,
 ['4f4533804b9074f6e4fb0105', '4bf58dd8d48988d13d941735','52e81612bcbc57f1066b7a46',
 '52e81612bcbc57f1066b7a45','4f4533814b9074f6e4fb0107','4f4533814b9074f6e4fb0106'], 5000)
cleaned_school_list = cleanVenueList(school_list)
possible_offices_c1_c2_c3_c4_c5_c6_c7 = [e[0] for e in cleaned_school_list]
possible_offices_c1_c2_c3_c4_c5_c6_c7_to_string = [str(e) for e in possible_offices_c1_c2_c3_c4_c5_c6_c7]
possible_offices_c1_c2_c3_c4_c5_c6_c7_coords = getOfficesCoords(possible_offices_c1_c2_c3_c4_c5_c6_c7, companies)




    
                                30% of the company have at least 1 child.
    ----------------------------------------------------------------------------------------------------
    

                            Please wait. We are collecting some information...
    


In [184]:
len(possible_offices_c1_c2_c3_c4_c5_c6_c7_coords)

4

In [200]:
possible_offices_c1_c2_c3_c4_c5_c6_c7_to_string

['52cdef7d4bab8bd675299115',
 '52cdef7d4bab8bd67529a0d1',
 '52cdef7c4bab8bd6752980be',
 '52cdef7c4bab8bd6752986aa']

In [201]:
# Filtered DF:
indexs = []
for i in range(len(df)):
    if df['_id'][i] in possible_offices_c1_c2_c3_c4_c5_c6_c7_to_string:
        indexs.append(i)
indexs

[669, 1696, 3232, 4956]

In [205]:
df_filtered = df.iloc[indexs]
df_filtered.reset_index(drop=True, inplace=True)
df_filtered

,_id,name,founded_year,category_code,deadpooled_year,total_money_raised_USD,num_offices,office_1_longitude,office_1_latitude,office_1_location,office_1_city,office_1_state_code,office_1_country_code
0,52cdef7c4bab8bd6752980be,Cyberhomes,NaN,web,NaN,0.0,1,-81.674745,30.318112,"{'type': 'Point', 'coordinates': [-81.674745, ...",Jacksonville,FL,USA
1,52cdef7c4bab8bd6752986aa,AudioMicro,2008.0,games_video,NaN,1250000.0,1,-118.422495,34.153588,"{'type': 'Point', 'coordinates': [-118.422495,...",Sherman Oaks,CA,USA
2,52cdef7d4bab8bd675299115,OTOY,2010.0,software,NaN,0.0,1,-118.424058,34.157610,"{'type': 'Point', 'coordinates': [-118.424058,...",Sherman Oaks,CA,USA
3,52cdef7d4bab8bd67529a0d1,Dukky,2008.0,analytics,NaN,1000000.0,1,-90.082202,29.973994,"{'type': 'Point', 'coordinates': [-90.082202, ...",Mandeville,LA,USA


In [208]:
rowindex

1

In [206]:
# Select one random row:
rowindex = random.choice(range(0,len(df_filtered)))
e = [df_filtered.iloc[rowindex][8], df_filtered.iloc[rowindex][7]]

In [207]:
e

[34.153588, -118.422495]

In [209]:
# Lat and long:
lat_long_starbucks = getLatLongVenue(cleaned_starbucks_list, df_filtered, rowindex)
distance_starbucks = getDistanceVenue(cleaned_starbucks_list, df_filtered, rowindex)

lat_long_vegan = getLatLongVenue(cleaned_vegan_list, df_filtered, rowindex)
distance_vegan = getDistanceVenue(cleaned_vegan_list, df_filtered, rowindex)
name_vegan = getNameVenue(cleaned_vegan_list, df_filtered, rowindex)

lat_long_party = getLatLongVenue(cleaned_party_list, df_filtered, rowindex)
distance_party = getDistanceVenue(cleaned_party_list, df_filtered, rowindex)
name_party = getNameVenue(cleaned_party_list, df_filtered, rowindex)

In [212]:
lat_long_party

[34.150357, -118.42032]

In [213]:
df_filtered.iloc[rowindex][0]

'52cdef7c4bab8bd6752986aa'

In [214]:
name_dist_lat_long_airport = []
for i in range(len(office_airports)):
    if str(office_airports[i][0]) == df_filtered.iloc[rowindex][0]:
        for j in range(1, len(office_airports[i])):
            name_dist_lat_long_airport.append(office_airports[i][j][0]) # name
            name_dist_lat_long_airport.append(office_airports[i][j][1]) # dist           
            name_dist_lat_long_airport.append(office_airports[i][j][2]) # lat
            name_dist_lat_long_airport.append(office_airports[i][j][3]) # long  

In [216]:
len(name_dist_lat_long_airport)/4 # total aeropuertos cercanos

4.0

In [217]:
lat_long_school = getLatLongVenue(cleaned_school_list, df_filtered, rowindex)
distance_school = getDistanceVenue(cleaned_school_list, df_filtered, rowindex)
name_school = getNameVenue(cleaned_school_list, df_filtered, rowindex)

In [219]:
near_startups = []
for i in range(len(startups_and_near_companies)):
    for company in startups_and_near_companies[i][1]:
        if company['name'] == df_filtered.iloc[rowindex][1]:
            near_startups.append(startups_and_near_companies[i][0])

In [224]:
# Output:
printoutput(df_filtered.iloc[rowindex][10], df_filtered.iloc[rowindex][12], inputyears,
inputmoney, distance_starbucks, name_vegan, distance_vegan, name_party, distance_party,
name_dist_lat_long_airport[0], round(name_dist_lat_long_airport[1],2), name_school, distance_school)




    ----------------------------------------------------------------------------------------------------
                            WE HAVE FOUND THE PERFECT LOCATION FOR YOUR COMPANY!           
    ----------------------------------------------------------------------------------------------------

        The perfect location for your business is in Sherman Oaks, USA.

        You don't have companies with more than 12 years in a radius of 2 KM (blue circle).

        The office is near successful tech startups that have raised at least 1000000 dollars.

        Your employees will find a Starbucks just 532 meters from the office.

        The vegan restaurant 'Hugo's Tacos' can be found just 893 meters from the office.

        Party mood? You will find the night club called 'Casita de Ramy Boo Sparkles' just 411 meters from the office.

        If you need to travel often, there is no problem. You have an airport (Bob Hope Airport) just 7.85 meters from the office.

        An

In [229]:
# Folium map:
tooltip = 'Click me!'
map_city = folium.Map(location = e, zoom_start=13)
folium.Circle(radius=2000,location=e,popup='Old companies free zone',color='#3186cc',
    fill=True,fill_color='#3186cc').add_to(map_city)
folium.Marker(lat_long_starbucks,radius=2,icon=folium.Icon(
    icon='coffee', prefix='fa',color='orange'),popup='<b>[Starbucks]</b>',
    tooltip=tooltip).add_to(map_city)
folium.Marker(lat_long_vegan,radius=2,icon=folium.Icon(
    icon='cutlery',color='green'),popup=f"<b>[Vegan restaurant]</b> '{name_vegan}'",
    tooltip=tooltip).add_to(map_city)
folium.Marker(lat_long_party,radius=2,icon=folium.Icon(
    icon='glass',color='purple'),popup=f"<b>[Night club]</b> '{name_party}'",
    tooltip=tooltip).add_to(map_city)
folium.Marker([name_dist_lat_long_airport[2],name_dist_lat_long_airport[3]],radius=2,icon=folium.Icon(
    icon='plane', prefix='fa',color='blue'),popup=f"<b>[Airport]</b> '{name_dist_lat_long_airport[0]}'",
    tooltip=tooltip).add_to(map_city)
folium.Marker(lat_long_school,radius=2,icon=folium.Icon(
    icon='graduation-cap', prefix='fa',color='gray'),popup=f"<b>[School]</b> '{name_school}'",
    tooltip=tooltip).add_to(map_city)
folium.Marker(e,radius=2,icon=folium.Icon(
    icon='briefcase', color='red'),popup='<b>Perfect location for your business</b>',
    tooltip=tooltip).add_to(map_city)
for startup in near_startups:
    category = re.sub("_"," ",startup[3].capitalize())
    folium.Marker([startup[6], startup[5]],radius=2,icon=folium.Icon(
        icon='building-o', prefix='fa',color='black'),
        popup=f"<b>[Startup]</b> {startup[1]}. Founded year: {int(startup[2])}. Category: {category}. Total money raised (USD): {int(startup[4])}.",
        tooltip=tooltip).add_to(map_city) 
map_city.save('./output/map3.html')
map_city

In [102]:
airports_df = pd.read_csv('./input/airports.csv', header=None,usecols=[1,2,3,4,6,7,12], 
                          names=['FacilityName', 'City', 'Country','3CharCode','Lat','Long','FacilityType'])

In [103]:
airports_df = airports_df[airports_df['FacilityType']=='airport']

In [104]:
airports_df.reset_index(drop=True, inplace=True)

In [105]:
import geopy.distance
distances = []
for e in possible_offices_c1_c2_coords:
    coords_1 = (e['coordinates'][1], e['coordinates'][0])
    dist = []
    for i in range(len(airports_df)):
        coords_2 = (airports_df.at[i,'Lat'],airports_df.at[i,'Long'])
        dist.append(geopy.distance.geodesic(coords_1, coords_2).km)
    distances.append(dist)    

In [140]:
office_airports = []
for i in range(len(distances)):
    aux = []
    aux.append(possible_offices_c1_c2[i])
    for j in range(len(distances[i])):
        if distances[i][j] <= 20:
            aux.append([airports_df.at[j,'FacilityName'],
                        distances[i][j],
                        airports_df.at[j,'Lat'],
                        airports_df.at[j,'Long']])
    office_airports.append(aux)

In [141]:
possible_offices_c1_c2_c3 = []
for e in office_airports:
    if len(e) >= 2:
        possible_offices_c1_c2_c3.append(e[0])

In [143]:
possible_offices_c1_c2_c3_to_string = [str(e) for e in possible_offices_c1_c2_c3]

In [146]:
possible_offices_c1_c2_c3_coords = getOfficesCoords(possible_offices_c1_c2_c3, companies)

In [ ]:
name_dist_lat_long_airport = []
for i in range(len(office_airports)):
    for j in range(1, len(office_airports[i])):
        if str(office_airports[i][0]) == df.iloc[rowindex][0]:
            name_dist_lat_long_airport.append(office_airports[i][j][0]) # name
            name_dist_lat_long_airport.append(office_airports[i][j][1]) # dist           
            name_dist_lat_long_airport.append(office_airports[i][j][2]) # lat
            name_dist_lat_long_airport.append(office_airports[i][j][3]) # long           